### Fine tuning Microsoft - Phi2 model on custom dataset
* Phi2 is smaller model compared to other models
* Using QLORA technique to do this


In [2]:
!pip install -q -U "torch==2.1.2" tensorboard
!pip install -q -U "transformers==4.36.2" "datasets==2.16.1" "accelerate==0.26.1" "bitsandbytes==0.42.0"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.
tensorflow 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.16.2 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.

In [3]:
!pip install -q -U git+https://github.com/huggingface/trl@a3c5b7178ac4f65569975efadc97db2f3749c65e
!pip install -q -U git+https://github.com/huggingface/peft@4a1559582281fc3c9283892caea8ccef1d6f5a4f

In [4]:
# disbale WanB
import os
os.environ['WANDB_DISABLED']="true"

In [34]:
# Imports
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig,AutoTokenizer,
TrainingArguments,Trainer,GenerationConfig,HfArgumentParser)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import transformers
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ·····································
Add token as git credential? (Y/n)  n


Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
# Loading dataset from huggingface
huggingface_dataset_name="neil-code/dialogsum-test"
dataset=load_dataset(huggingface_dataset_name)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


Generating validation split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


Generating test split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1999
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 499
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 499
    })
})

In [7]:
# looking at dataset
dataset['train'][2]

{'id': 'train_2',
 'dialogue': "#Person1#: Excuse me, did you see a set of keys?\n#Person2#: What kind of keys?\n#Person1#: Five keys and a small foot ornament.\n#Person2#: What a shame! I didn't see them.\n#Person1#: Well, can you help me look for it? That's my first time here.\n#Person2#: Sure. It's my pleasure. I'd like to help you look for the missing keys.\n#Person1#: It's very kind of you.\n#Person2#: It's not a big deal.Hey, I found them.\n#Person1#: Oh, thank God! I don't know how to thank you, guys.\n#Person2#: You're welcome.",
 'summary': "#Person1#'s looking for a set of keys and asks for #Person2#'s help to find them.",
 'topic': 'find keys'}

#### Creating bnb config- we need configuration class to specify which kind of quantization we need, using bitsandbytes(bnb) for making this configuration so that LLM can be loaded in 4 bits so that memory utilization is less and model can be loaded on small devices

#### Load Base Model Phi2 in 4 bit Quantization using bnb config

In [8]:
model_name = 'microsoft/phi-2'

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config, 
)

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at microsoft/phi-2 were not used when initializing PhiForCausalLM: ['model.layers.26.self_attn.k_proj.weight', 'model.layers.7.self_attn.k_proj.weight', 'model.layers.27.self_attn.k_proj.bias', 'model.layers.6.self_attn.q_proj.weight', 'model.layers.13.self_attn.q_proj.bias', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.19.self_attn.q_proj.weight', 'model.layers.20.self_attn.q_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.12.self_attn.v_proj.weight', 'model.layers.21.self_attn.q_proj.weight', 'model.layers.23.self_attn.k_proj.weight', 'model.layers.5.self_attn.q_proj.bias', 'model.layers.12.self_attn.q_proj.weight', 'model.layers.15.self_attn.v_proj.weight', 'model.layers.6.self_attn.v_proj.weight', 'model.layers.20.self_attn.v_proj.weight', 'model.layers.9.self_attn.k_proj.bias', 'model.layers.30.self_attn.k_proj.weight', 'model.layers.29.self_attn.q_proj.weight', 'model.layers.30.self_attn.q_proj.bias', 'model.layers

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

#### Setting tokenizer - using left padding to save memory

In [9]:
# Setting tokenizer
tokenizer=AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,
                                        padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token=tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Creating a funtion to track GPU utilization at regular intervals

In [10]:
from pynvml import *
def print_gpu_utilization():
    nvmlInit()
    handle=nvmlDeviceGetHandleByIndex(0)
    info=nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

In [11]:
print_gpu_utilization()

GPU memory occupied: 2229 MB.


#### Defining evaluation tokenizer-

In [12]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_name, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=100, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.1,num_beams=1,top_p=0.95,).to('cpu')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Testing model with zero shot inferencing

In [13]:
%%time
from transformers import set_seed
seed=42
set_seed(seed)
index=10
prompt=dataset['test'][index]['dialogue']
summary=dataset['train'][index]['summary']
formatted_prompt=f"Instruct: Summarize the following conversation.\n{prompt}\nOutput:\n"
res=gen(model,formatted_prompt,100,)
output=res[0].split('Output:\n')[1]
sep_line='*'*100
print(sep_line)
print(f'Input Prompt:\n{formatted_prompt}\n')
print(sep_line)
print(f'Baseline Human summary:\n{summary}\n')
print(sep_line)
print(f'Model Generated Zero Shot"\n{output}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


****************************************************************************************************
Input Prompt:
Instruct: Summarize the following conversation.
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
Output:


****************************************************************************************************
Baseline Human summary:
#Person1# asks #Person2# to do a f

#### Preprocess dataset

In [14]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output'), concatenate them
    """
    INTRO_BLURB="Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY="### Instruct: Summarize the below conversation."
    RESPONSE_KEY="### Output:"
    END_KEY="### End"
    blurb=f"\n{INTRO_BLURB}"
    instruction=f"{INSTRUCTION_KEY}"
    input_context=f"{sample['dialogue']}" if sample["dialogue"] else None
    response=f"{RESPONSE_KEY}\n{sample['summary']}"
    end=f"{END_KEY}"
    parts=[part for part in [blurb,instruction,input_context,response,end] if part]
    formatted_prompt="\n\n".join(parts)
    sample["text"]=formatted_prompt
    return sample

In [15]:
def get_max_length(model):
    conf=model.config
    max_length=None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length=getattr(model.config,length_setting,None)
        if max_length:
            print(f"FOund max length:{max_length}")
            break
    if not max_length:
        max_length=1024
        print("Using default max length")
    return max_length

def preprocess_batch(batch,tokenizer,max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(batch["text"],max_length=max_length,truncation=True)


#### Format and tokenize the dataset to make it training ready

In [16]:
from functools import partial
def preprocess_dataset(tokenizer:AutoTokenizer,max_length:int,seed,dataset):
    print("Preprocessing Dataset\n")
    dataset=dataset.map(create_prompt_formats)
    preprocessing_function=partial(preprocess_batch,max_length=max_length,tokenizer=tokenizer)
    dataset=dataset.map(preprocessing_function,batched=True,remove_columns=['id','topic','dialogue','summary'])
    dataset=dataset.filter(lambda sample:len(sample["input_ids"])<max_length)
    dataset=dataset.shuffle(seed=seed)
    return dataset

In [17]:
print_gpu_utilization()

GPU memory occupied: 2579 MB.


#### Finally preprocess dataset

In [18]:
max_length=get_max_length(model)
print(max_length)

FOund max length:2048
2048


In [19]:
train_dataset=preprocess_dataset(tokenizer,max_length,seed,dataset['train'])
eval_dataset=preprocess_dataset(tokenizer,max_length,seed,dataset['validation'])
print("Shapes of datasets\n")
print(f"Training dataset:{train_dataset.shape}")
print(f"Validation dataset:{eval_dataset.shape}")

Preprocessing Dataset



Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1999 [00:00<?, ? examples/s]

Preprocessing Dataset



Map:   0%|          | 0/499 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

Filter:   0%|          | 0/499 [00:00<?, ? examples/s]

Shapes of datasets

Training dataset:(1999, 3)
Validation dataset:(499, 3)


In [20]:
print(train_dataset)

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 1999
})


#### Setup PEFT and LORA model for fine tuning
* PEFT is a form of instruction fine tuning more efficient than full fine tuning, it comes in2 variants - LoRA and QLoRA, LoRA is used here, it used lower resources.
* Doing LoRA finetuning results in an unchanged original LLM & smaller LoRA adapter, during inference time LoRA adapter must be combined with original LLM

#### Creating a method to calculate number of trainable parameters

In [24]:
def trainable_model_parameters_count(model):
    trainable_model_params=0
    all_model_params=0
    for _,param in model.named_parameters():
        all_model_params+=param.numel()
        if param.requires_grad:
            trainable_model_params+=param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [25]:
print(trainable_model_parameters_count(model))

trainable model parameters: 262364160
all model parameters: 1521392640
percentage of trainable model parameters: 17.24%


In [27]:
print(model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (query_key_value): Linear4bit(in_features=2560, out_features=7680, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_

In [31]:
from peft import LoraConfig,get_peft_model,prepare_model_for_kbit_training
config=LoraConfig(r=32,lora_alpha=32,target_modules=[
    'q_proj','k_proj','v_proj','dense'
],bias='none',lora_dropout=0.07,task_type="CAUSAL_LM")
#Activating gradient checkpointing to reduce memory usage during fine-tuning
model.gradient_checkpointing_enable()
#Using the prepare_model_for_kbit_training method from PEFT
model=prepare_model_for_kbit_training(model)
peft_model=get_peft_model(model, config)

#### Now PEFT configuration is setup, find number of trainable parameters in this model

In [32]:
print(trainable_model_parameters_count(peft_model))

trainable model parameters: 5242880
all model parameters: 1526635520
percentage of trainable model parameters: 0.34%


#### We can see in PEFT number of trainable parameters is much less compared to original model

In [33]:
print(peft_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2560)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (query_key_value): Linear4bit(in_features=2560, out_features=7680, bias=True)
              (dense): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.07, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
             

#### Define training arguments and setup trainer

In [40]:
output_dir='./peft-dialogue-summary-training/final-checkpoint'
peft_training_args=TrainingArguments(output_dir=output_dir,warmup_steps=1,
                        per_device_train_batch_size=1,gradient_accumulation_steps=5,
                        max_steps=200,learning_rate=2e-4,optim="paged_adamw_8bit",
                        logging_steps=25,logging_dir="./logs",save_strategy="steps",
                        save_steps=25,evaluation_strategy="steps",eval_steps=25,
                        do_eval=True,gradient_checkpointing=True,report_to=None,
                                    overwrite_output_dir='True',group_by_length=True)
peft_model.config.use_cache=False
peft_trainer=transformers.Trainer(model=peft_model,train_dataset=train_dataset,
                                 eval_dataset=eval_dataset,args=peft_training_args,
            data_collator=transformers.DataCollatorForLanguageModeling(tokenizer,mlm=False))

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [41]:
peft_training_args.device

device(type='cuda', index=0)

In [42]:
print_gpu_utilization()

GPU memory occupied: 10533 MB.


In [43]:
peft_trainer.train()

Step,Training Loss,Validation Loss
25,3.656100,3.294434
50,2.850300,3.303666
75,3.488400,3.147907
100,2.794800,3.244513
125,3.319400,3.124656
150,2.679900,3.206429
175,3.390600,3.099191
200,2.777800,3.096688


Checkpoint destination directory ./peft-dialogue-summary-training/final-checkpoint/checkpoint-25 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. 

TrainOutput(global_step=200, training_loss=3.1196464920043945, metrics={'train_runtime': 2603.2653, 'train_samples_per_second': 0.384, 'train_steps_per_second': 0.077, 'total_flos': 4535368716257280.0, 'train_loss': 3.1196464920043945, 'epoch': 0.5})

In [44]:
print_gpu_utilization()

GPU memory occupied: 10543 MB.


#### Evaluate the model

In [45]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
base_model_id="microsoft/phi-2"
base_model=AutoModelForCausalLM.from_pretrained(base_model_id,device_map='auto',
                                quantization_config=bnb_config,trust_remote_code=True,use_auth_token=True)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [46]:
eval_tokenizer=AutoTokenizer.from_pretrained(base_model_id,add_bos_token=True,trust_remote_code=True,use_fast=False)
eval_tokenizer.pad_token=eval_tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [48]:
from peft import PeftModel
ft_model=PeftModel.from_pretrained(base_model,"/kaggle/working/peft-dialogue-summary-training/final-checkpoint/checkpoint-200",torch_dtype=torch.float16,is_trainable=False)

In [49]:
%%time
from transformers import set_seed
set_seed(seed)
index=10
dialogue=dataset['test'][index]['dialogue']
summary=dataset['test'][index]['summary']
prompt=f"Instruct: Summarize the following conversation.\n{dialogue}\nOutput:\n"
peft_model_res=gen(ft_model,prompt,100,)
peft_model_output=peft_model_res[0].split('Output:\n')[1]
prefix,success,result=peft_model_output.partition('#End')
sep_line="*"*100
print(sep_line)
print(f'INPUT PROMPT:\n{prompt}')
print(sep_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(sep_line)
print(f'PEFT MODEL:\n{prefix}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


****************************************************************************************************
INPUT PROMPT:
Instruct: Summarize the following conversation.
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
Output:

****************************************************************************************************
BASELINE HUMAN SUMMARY:
#Person1# attends Brian's birthday 

#### The above evaluation is just qualitative without using any metric , now trying to evaluate model quantitatively using ROUGE metric
* Doing this only for few samples only to save time(7 samples)

In [50]:
original_model=AutoModelForCausalLM.from_pretrained(base_model_id,device_map="auto",
                quantization_config=bnb_config,trust_remote_code=True,use_auth_token=True)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [52]:
dialogues=dataset['test'][0:7]['dialogue']
dataset_baseline_summaries=dataset['test'][0:7]['summary']
original_model_summaries=[]
peft_model_summaries=[]
for idx,dialogue in enumerate(dialogues):
    baseline_text_output=dataset_baseline_summaries[idx]
    prompt=f"Instruct: Summarize the following conversation:\n{dialogue}\nOutput:\n"
    original_model_res=gen(original_model,prompt,100,)
    original_model_text_output=original_model_res[0].split('Output:\n')[1]
    peft_model_res=gen(ft_model,prompt,100,)
    peft_model_output=peft_model_res[0].split('Output:\n')[1]
    peft_model_text_output,success,result=peft_model_output.partition('#End')
    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)
zipped_summaries=list(zip(dataset_baseline_summaries,original_model_summaries,peft_model_summaries))
df=pd.DataFrame(zipped_summaries,columns=['dataset_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])
df

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

,dataset_baseline_summaries,original_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,"Person 1: Ms. Dawson, I need you to take a dic...","Person 1: Ms. Dawson, I need you to take a dic..."
1,In order to prevent employees from wasting tim...,"Person 1: Ms. Dawson, I need you to take a dic...","Person 1: Ms. Dawson, I need you to take a dic..."
2,Ms. Dawson takes a dictation for #Person1# abo...,"Person 1: Ms. Dawson, I need you to take a dic...","Person 1: Ms. Dawson, I need you to take a dic..."
3,#Person2# arrives late because of traffic jam....,Person1 and Person2 are discussing the traffic...,Person1 and Person2 are discussing the traffic...
4,#Person2# decides to follow #Person1#'s sugges...,Person1 and Person2 are discussing the traffic...,Person1 and Person2 are discussing the traffic...
5,#Person2# complains to #Person1# about the tra...,Person1 and Person2 are discussing the traffic...,Person1 and Person2 are discussing the traffic...
6,#Person1# tells Kate that Masha and Hero get d...,Kate: Masha and Hero are getting divorced afte...,Kate: Masha and Hero are getting divorced afte...


In [53]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=824b7dce9c9f9039267f8de38fef4dd8582a8c4487961ae1c32d728a51275113
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [55]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.0 MB/s eta 0:00:00


In [57]:
import evaluate
rouge = evaluate.load('rouge')
original_model_results=rouge.compute(
    predictions=original_model_summaries,
    references=dataset_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)
peft_model_results=rouge.compute(
    predictions=peft_model_summaries,
    references=dataset_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)
print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)


ORIGINAL MODEL:
{'rouge1': 0.27667979627279304, 'rouge2': 0.08875241694518803, 'rougeL': 0.19135388883743298, 'rougeLsum': 0.21150262085032032}
PEFT MODEL:
{'rouge1': 0.3040309821949655, 'rouge2': 0.10936087174375382, 'rougeL': 0.20278568805863828, 'rougeLsum': 0.2368705591597158}


In [58]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")
improvement=(np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 2.74%
rouge2: 2.06%
rougeL: 1.14%
rougeLsum: 2.54%
